In [107]:
import geopandas as gpd
import os
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

## Load Roads

In [11]:
MAPA_PATH = "../../../data/Mapa/Ben Gurion University/Gisrael/"

In [19]:
roads_df = gpd.read_file(MAPA_PATH + 'is_str.shp')

In [92]:
roads_df = roads_df.set_index('UserID')

In [93]:
roads_df.head(1)

,street,FromLeft,ToLeft,FromRight,ToRight,StreetCode,ObjID,FJunction,TJunction,Length,...,Status,RoadFuncti,Flag,Identifier,Roaddirect,CLEARSTREE,REVERSTREE,CLEARCITYN,REVERCITYN,geometry
UserID,,,,,,,,,,,,,,,,,,,,,
1004561,None,0,0,0,0,None,1,947733,947736,64.569179,...,Operative,4x4 Road,5,1012284,222222,None,None,àéìú,úìéà,"LINESTRING (194289.114 388953.679, 194277.535 ..."


In [94]:
print('Number of edges (links) in networks: ' + str(roads_df.shape[0]))

Number of edges (links) in networks: 647504


In [95]:
unique_junctions = roads_df['FJunction'].append(roads_df['TJunction']).unique()

In [96]:
print('Number of nodes (junctions) in networks: ' + str(unique_junctions.shape[0]))

Number of nodes (junctions) in networks: 489961


In [97]:
num_orogins = roads_df['FJunction'].unique().shape[0]
print('Number of unique link origins: ' + str(num_orogins))
print('This means an average out-degree of ' + str(roads_df.shape[0] / num_orogins))

Number of unique link origins: 438984
This means an average out-degree of 1.4750059227671168


In [98]:
num_targets = roads_df['TJunction'].unique().shape[0]
print('Number of unique link targets: ' + str(num_targets))
print('This means an average in-degree of ' + str(roads_df.shape[0] / num_targets))

Number of unique link targets: 456354
This means an average in-degree of 1.418863426199836


### Get Edges (based on one-way and two-way roads) 

In [120]:
roads_df.columns

Index(['street', 'FromLeft', 'ToLeft', 'FromRight', 'ToRight', 'StreetCode',
       'ObjID', 'FJunction', 'TJunction', 'Length', 'RoadType', 'OneWay',
       'F_ZLev', 'T_ZLev', 'FLanes', 'TLanes', 'Intended', 'cityname',
       'CityCode', 'Seconds', 'FromSpeedL', 'ToSpeedL', 'AprxSpeedL',
       'Autonomy', 'Regulation', 'RoadNameMZ', 'DirectionM', 'Status',
       'RoadFuncti', 'Flag', 'Identifier', 'Roaddirect', 'CLEARSTREE',
       'REVERSTREE', 'CLEARCITYN', 'REVERCITYN', 'geometry'],
      dtype='object')

In [162]:
shape_edges_df = roads_df[['ObjID', 'FJunction', 'TJunction', 'RoadType', 
                     'OneWay', 'FromSpeedL', 'ToSpeedL', 
                     'AprxSpeedL', 'Seconds', 'Length', 'Roaddirect', 'DirectionM']]
shape_edges_df = shape_edges_df.reset_index()

In [163]:
shape_edges_df.head(2)

,UserID,ObjID,FJunction,TJunction,RoadType,OneWay,FromSpeedL,ToSpeedL,AprxSpeedL,Seconds,Length,Roaddirect,DirectionM
0,1004561,1,947733,947736,254,2,10,10,10,69.734908,64.569179,222222,None
1,1059914,2,950536,983821,10,2,90,90,90,2.249966,49.999104,222222,B


In [164]:
edges_df = pd.DataFrame()
edges_df.columns = edges_df.columns

In [165]:
shape_edges_df['OneWay'].unique()

array(['2', 'ft', 'N', 'tf'], dtype=object)

In [166]:
def create_edges(e):
    road_type = e['OneWay']

    if road_type == 'N':
        return
    elif road_type == 'ft' or road_type == '2':
        edges_df.append(e)
    elif road_type == 'tf' or road_type == '2':
        frm = e['FJunction']
        to = e['TJunction']
        tf_edge = e
        tf_edge['FJunction'] = to
        tf_edge['TJunction'] = frm
        edges_df.append(tf_edge)
        

In [167]:
shape_edges_df.apply(lambda x: create_edges(x), axis=1)

0         None
1         None
2         None
3         None
4         None
          ... 
647499    None
647500    None
647501    None
647502    None
647503    None
Length: 647504, dtype: object

In [168]:
# Sanity check
number_of_edges = edges.shape[0]
number_of_ft = roads_df[roads_df['OneWay'] == 'ft'].shape[0]
number_of_tf = roads_df[roads_df['OneWay'] == 'tf'].shape[0]
number_of_two_way = roads_df[roads_df['OneWay'] == '2'].shape[0]

assert number_of_edges == (number_of_ft + number_of_tf + number_of_two_way*2)

NameError: name 'edges' is not defined

### First Stage - without turn restrictions

In [99]:
# Get out degrees
unique_origins_count_df = roads_df['FJunction'].value_counts()
unique_origins_df = pd.DataFrame()
unique_origins_df['id'] = unique_origins_count_df.index
unique_origins_df['out_degree'] = unique_origins_count_df.values

In [100]:
# Get in degrees
unique_targets_count_df = roads_df['TJunction'].value_counts()
unique_targets_df = pd.DataFrame()
unique_targets_df['id'] = unique_targets_count_df.index
unique_targets_df['in_degree'] = unique_targets_count_df.values

In [101]:
# Create basic nodes with in and out degrees
nodes_df = pd.DataFrame()
nodes_df['id'] = pd.Series(unique_junctions)
nodes_df = nodes_df.merge(unique_origins_df, how='left', on='id')
nodes_df = nodes_df.merge(unique_targets_df, how='left', on='id')
nodes_df = nodes_df.fillna(0)

In [102]:
nodes_df.head()

,id,out_degree,in_degree
0,947733,1.0,1.0
1,950536,1.0,1.0
2,17832,2.0,1.0
3,912962,2.0,0.0
4,1141238,1.0,1.0


In [104]:
nodes_df.shape

(489961, 3)

__Find "middle" nodes (one link enters, one link leaves)__ <br>
Next we'll need to see if "both sides" of the node are similar: same type (one-way/two-way), same speed

In [109]:
middle_nodes_df = nodes_df[nodes_df['in_degree'] == 1][nodes_df['out_degree'] == 1]

In [115]:
middle_nodes_df.head()

,id,out_degree,in_degree
0,947733,1.0,1.0
1,950536,1.0,1.0
4,1141238,1.0,1.0
6,929680,1.0,1.0
7,983803,1.0,1.0


In [116]:
roads_df.columns

Index(['street', 'FromLeft', 'ToLeft', 'FromRight', 'ToRight', 'StreetCode',
       'ObjID', 'FJunction', 'TJunction', 'Length', 'RoadType', 'OneWay',
       'F_ZLev', 'T_ZLev', 'FLanes', 'TLanes', 'Intended', 'cityname',
       'CityCode', 'Seconds', 'FromSpeedL', 'ToSpeedL', 'AprxSpeedL',
       'Autonomy', 'Regulation', 'RoadNameMZ', 'DirectionM', 'Status',
       'RoadFuncti', 'Flag', 'Identifier', 'Roaddirect', 'CLEARSTREE',
       'REVERSTREE', 'CLEARCITYN', 'REVERCITYN', 'geometry'],
      dtype='object')

In [117]:
middle_nodes_df = middle_nodes_df.merge(roads_df, how='left', left_on='id', right_on='FJunction')

In [118]:
middle_nodes_df.head(2)

,id,out_degree,in_degree,street,FromLeft,ToLeft,FromRight,ToRight,StreetCode,ObjID,...,Status,RoadFuncti,Flag,Identifier,Roaddirect,CLEARSTREE,REVERSTREE,CLEARCITYN,REVERCITYN,geometry
0,947733,1.0,1.0,None,0,0,0,0,None,1,...,Operative,4x4 Road,5,1012284,222222,None,None,àéìú,úìéà,"LINESTRING (194289.114 388953.679, 194277.535 ..."
1,950536,1.0,1.0,40,0,0,0,0,None,2,...,Operative,Major Road,1,0,222222,40,40,îöôäøîåï,ïåîø äôöî,"LINESTRING (187484.184 501340.215, 187506.545 ..."


In [119]:
middle_nodes_df = middle_nodes_df.merge(roads_df, how='left', left_on='id', right_on='TJunction', suffixes=('_before','_after'))